In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests
import zipfile
import os
import glob
from datetime import datetime


In [2]:
def app_log(nv,msg):
    print(" "*nv + "[{}] INFO - {}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"),msg))
    

In [6]:

app_log(0,"Limpando ambiente")

if not os.path.exists("database\db"):
    os.makedirs("database\db")

if os.path.exists("database\db\dbo.db"):
  os.remove("database\db\dbo.db")

app_log(0,"Criando banco de dados")
conn = sqlite3.connect("database\db\dbo.db")
conn.execute("ATTACH DATABASE 'database\db\dbo.db' AS 'dbo';")
cur  = conn.cursor()

app_log(0,"Criando tabelas no banco de dados")
sql_file = open(r"database\ddl\create_tables.sql",encoding="ISO-8859-1")
sql_as_string = sql_file.read()
sql_file.close()
cur.executescript(sql_as_string)

conn.commit()
conn.close()


[17/10/2023 17:21:04] INFO - Limpando ambiente
[17/10/2023 17:21:04] INFO - Criando banco de dados
[17/10/2023 17:21:04] INFO - Criando tabelas no banco de dados


In [7]:
conn = sqlite3.connect("database\db\dbo.db")
dir_list = glob.glob('data/*.csv')
app_log(0,"Carga de dados no banco de dados")
app_log(1,"Tabelas Dimensao ")
for arq in filter(lambda file: "Dim" in file,dir_list):
    cursor = conn.execute("select * from {}".format(arq[5:-4]))
    colunas = [description[0] for description in cursor.description]
    df = pd.read_csv(arq,sep="|",
                     encoding="utf16",header=None,names=colunas,lineterminator ='\n',
                     keep_default_na=False)
    df = df.replace(r'\r',' ', regex=True) 
    app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
    df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

app_log(1,"Tabelas Fato")
for arq in filter(lambda file: "Fact" in file,dir_list):
    cursor = conn.execute("select * from {}".format(arq[5:-4]))
    colunas = [description[0] for description in cursor.description]
    df = pd.read_csv(arq,sep="|",
                     encoding="utf16",header=None,names=colunas,lineterminator ='\n',
                     keep_default_na=False)
    df = df.replace(r'\r',' ', regex=True) 
    app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
    df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

app_log(1,"Demais Tabelas ")
for arq in filter(lambda file: "Fact" not in file and "Dim" not in file ,dir_list):
    try:
        cursor = conn.execute("select * from {}".format(arq[5:-4]))
        colunas = [description[0] for description in cursor.description]
        df = pd.read_csv(arq,sep="|",
                        encoding="utf16",header=None,names=colunas,lineterminator ='\n',
                        keep_default_na=False)
        df = df.replace(r'\r',' ', regex=True) 
        app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
        df.to_sql(arq[5:-4],conn,if_exists='append',index=False)
    except:
        df = pd.read_csv(arq,sep="|",
                        encoding="utf16",header=None,lineterminator ='\n',
                        keep_default_na=False)
        df = df.replace(r'\r',' ', regex=True) 
        app_log(2,"Carregando {} - {} linhas".format(arq[5:-4],df.shape[0]))
        df.to_sql(arq[5:-4],conn,if_exists='append',index=False)

conn.close()

[17/10/2023 17:21:05] INFO - Carga de dados no banco de dados
 [17/10/2023 17:21:05] INFO - Tabelas Dimensao 
  [17/10/2023 17:21:05] INFO - Carregando DimAccount - 99 linhas
  [17/10/2023 17:21:05] INFO - Carregando DimCurrency - 105 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimCustomer - 18484 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimDate - 3652 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimDepartmentGroup - 7 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimEmployee - 296 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimGeography - 655 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimOrganization - 14 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimProduct - 606 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimProductCategory - 4 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimProductSubcategory - 37 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimPromotion - 16 linhas
  [17/10/2023 17:21:06] INFO - Carregando DimReseller - 701 linhas


In [8]:
conn = sqlite3.connect("database\db\dbo.db")
sql_query = """SELECT * FROM FactInternetSales;"""
df = pd.read_sql(sql_query,conn)
conn.close()
df.T

,0,1,2,3,4,5,6,7,8,9,...,60388,60389,60390,60391,60392,60393,60394,60395,60396,60397
ProductKey,310,346,346,336,346,311,310,351,344,312,...,491,225,528,537,214,485,225,485,486,225
OrderDateKey,20101229,20101229,20101229,20101229,20101229,20101230,20101230,20101230,20101230,20101231,...,20140128,20140128,20140128,20140128,20140128,20140128,20140128,20140128,20140128,20140128
DueDateKey,20110110,20110110,20110110,20110110,20110110,20110111,20110111,20110111,20110111,20110112,...,20140209,20140209,20140209,20140209,20140209,20140209,20140209,20140209,20140209,20140209
ShipDateKey,20110105,20110105,20110105,20110105,20110105,20110106,20110106,20110106,20110106,20110107,...,20140204,20140204,20140204,20140204,20140204,20140204,20140204,20140204,20140204,20140204
CustomerKey,21768,28389,25863,14501,11003,27645,16624,11005,11011,27621,...,18749,18749,15251,15251,15251,15868,15868,18759,18759,18759
PromotionKey,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CurrencyKey,19,39,100,100,6,100,6,6,6,100,...,100,100,100,100,100,100,100,100,100,100
SalesTerritoryKey,6,7,1,4,9,4,9,9,9,4,...,6,6,6,6,6,6,6,6,6,6
SalesOrderNumber,SO43697,SO43698,SO43699,SO43700,SO43701,SO43702,SO43703,SO43704,SO43705,SO43706,...,SO75120,SO75120,SO75121,SO75121,SO75121,SO75122,SO75122,SO75123,SO75123,SO75123
SalesOrderLineNumber,1,1,1,1,1,1,1,1,1,1,...,2,3,1,2,3,1,2,1,2,3
